In [2]:
train_size = 47500

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import itertools
import time
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
%matplotlib inline

In [4]:
np.set_printoptions(precision=9)
np.set_printoptions(linewidth=np.inf)

In [5]:
X = np.load('../../X_train.npz')['arr_0']
Y = np.load('../../Y_train.npz')['arr_0']
TX = np.load('../../X_test.npz')['arr_0']

In [5]:
XTX = np.concatenate((X, TX), axis=0)

In [6]:
XTX_square = XTX ** 2
XTX_cumsum = np.concatenate((np.cumsum(XTX[:, :5000], axis=1), np.cumsum(XTX[:, 5000:], axis=1)), axis=1)
XTX_all = np.concatenate((XTX, XTX_square, XTX_cumsum), axis=1)

In [7]:
XTX_all.shape

(50000, 30000)

In [8]:
X, X_test = XTX_all[:train_size], XTX_all[train_size:]
print(X.shape, X_test.shape)

(47500, 30000) (2500, 30000)


In [6]:
# calculate type 1 error
def err1(y, y_pred):
    return ('t1e', np.sum(1.0 * np.abs(y_pred - y)) / len(y_pred), False)

# calculate type 2 errr
def err2(y, y_pred):
    return ('t2e', np.sum(np.abs(y_pred - y) / y) / len(y_pred), False)

In [10]:
scorer = {'t1':err1, 't2':err2}

In [12]:
# select best params
y_id = 0
idx = np.random.permutation(47500)[:1000]
X_tmp, y_tmp = X[idx], Y[idx]
params = {'boosting_type':['gbdt', 'dart'],
          'num_leaves':[31],
          'learning_rate':[0.1],
          'n_estimators':[20],
          'n_jobs':[23]}

t1_err = np.Inf
t2_err = np.Inf
t1_best_params = {}
t2_best_params = {}
keys, values = zip(*params.items())
for v in itertools.product(*values):
    param = dict(zip(keys, v))
    print(param)
    t1_errr = 0
    t2_errr = 0
    kf = KFold(n_splits=5, shuffle=False)
    for train_idx, val_idx in kf.split(X_tmp):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = Y[train_idx, y_id], Y[val_idx, y_id]
        
        model = lgb.LGBMRegressor(**param)
        model.fit(X_train, y_train)
        t1_errr += err1(model.predict(X_val), y_val)[1]
        t2_errr += err2(model.predict(X_val), y_val)[1]
    
    if t1_errr / 5 < t1_err:
        t1_best_params = param
        t1_err = t1_errr / 5
    if t2_errr / 5 < t2_err:
        t2_best_params = param
        t2_err = t2_errr / 5

{'boosting_type': 'gbdt', 'num_leaves': 31, 'learning_rate': 0.1, 'n_estimators': 20, 'n_jobs': 23}
{'boosting_type': 'dart', 'num_leaves': 31, 'learning_rate': 0.1, 'n_estimators': 20, 'n_jobs': 23}


In [23]:
print(t1_best_params)
print(t2_best_params)

{'boosting_type': 'gbdt', 'num_leaves': 31, 'learning_rate': 0.1, 'n_estimators': 20, 'n_jobs': 23}
{'boosting_type': 'gbdt', 'num_leaves': 31, 'learning_rate': 0.1, 'n_estimators': 20, 'n_jobs': 23}


In [11]:
params = {'boosting_type':'gbdt', 'num_leaves':31, 'learning_rate':0.05, 'n_estimators':1000, 'n_jobs':23}

In [12]:
ty = []
vy = []
feature_imp = []

for y_id in range(3):
    kf = KFold(n_splits=5, shuffle=False)
    tty = []
    tvy = []
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = Y[train_idx, y_id], Y[val_idx, y_id]
        
        model = lgb.LGBMRegressor(**params)
        model.fit(X_train, y_train,  
                  eval_set=[(X_val, y_val)], 
                  eval_metric=scorer['t1'], 
                  early_stopping_rounds=5)
        feature_imp.append(model.feature_importances_)
        
        tvy.append(model.predict(X_val))
        tty.append(model.predict(X_test))
    vy.append(np.concatenate((tvy[0], tvy[1], tvy[2], tvy[3], tvy[4]), axis=0))
    ty.append(np.mean(np.array(tty), axis=0))

ty = np.array(ty).transpose()
vy = np.array(vy).transpose()
df = pd.DataFrame(ty)
df.to_csv('t1_test.csv', index=False, header=False)
df = pd.DataFrame(vy)
df.to_csv('t1_val.csv', index=False, header=False)
feature_imp = np.array(feature_imp).transpose()
df = pd.DataFrame(feature_imp)
df.to_csv('feature_imp.csv', index=False, header=False)

[1]	valid_0's l2: 0.0797814	valid_0's t1e: 0.245371
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 0.0750526	valid_0's t1e: 0.23809
[3]	valid_0's l2: 0.070744	valid_0's t1e: 0.2312
[4]	valid_0's l2: 0.0668024	valid_0's t1e: 0.224759
[5]	valid_0's l2: 0.0631663	valid_0's t1e: 0.218613
[6]	valid_0's l2: 0.0598537	valid_0's t1e: 0.21285
[7]	valid_0's l2: 0.0568177	valid_0's t1e: 0.20739
[8]	valid_0's l2: 0.0540238	valid_0's t1e: 0.202198
[9]	valid_0's l2: 0.0515034	valid_0's t1e: 0.197334
[10]	valid_0's l2: 0.0491692	valid_0's t1e: 0.192727
[11]	valid_0's l2: 0.0469983	valid_0's t1e: 0.188265
[12]	valid_0's l2: 0.045037	valid_0's t1e: 0.184187
[13]	valid_0's l2: 0.0431888	valid_0's t1e: 0.180176
[14]	valid_0's l2: 0.0415124	valid_0's t1e: 0.176464
[15]	valid_0's l2: 0.0399797	valid_0's t1e: 0.172929
[16]	valid_0's l2: 0.038583	valid_0's t1e: 0.169673
[17]	valid_0's l2: 0.0373024	valid_0's t1e: 0.166596
[18]	valid_0's l2: 0.0360864	valid_0's t1e: 0.163592
[1

[154]	valid_0's l2: 0.0175187	valid_0's t1e: 0.099109
[155]	valid_0's l2: 0.0175153	valid_0's t1e: 0.0990934
[156]	valid_0's l2: 0.0175194	valid_0's t1e: 0.0991061
[157]	valid_0's l2: 0.0175161	valid_0's t1e: 0.0990936
[158]	valid_0's l2: 0.017511	valid_0's t1e: 0.0990745
[159]	valid_0's l2: 0.0175026	valid_0's t1e: 0.0990297
[160]	valid_0's l2: 0.0174926	valid_0's t1e: 0.0989822
[161]	valid_0's l2: 0.0174936	valid_0's t1e: 0.0989892
[162]	valid_0's l2: 0.0174924	valid_0's t1e: 0.0989746
[163]	valid_0's l2: 0.0174895	valid_0's t1e: 0.0989693
[164]	valid_0's l2: 0.0174901	valid_0's t1e: 0.0989739
[165]	valid_0's l2: 0.0174938	valid_0's t1e: 0.0989794
[166]	valid_0's l2: 0.0174893	valid_0's t1e: 0.0989588
[167]	valid_0's l2: 0.0174899	valid_0's t1e: 0.0989554
[168]	valid_0's l2: 0.0174873	valid_0's t1e: 0.0989517
[169]	valid_0's l2: 0.0174858	valid_0's t1e: 0.0989488
[170]	valid_0's l2: 0.0174803	valid_0's t1e: 0.0989298
[171]	valid_0's l2: 0.0174766	valid_0's t1e: 0.0989256
[172]	valid_

[27]	valid_0's l2: 0.0282841	valid_0's t1e: 0.14242
[28]	valid_0's l2: 0.0277417	valid_0's t1e: 0.140694
[29]	valid_0's l2: 0.0272546	valid_0's t1e: 0.139151
[30]	valid_0's l2: 0.0267761	valid_0's t1e: 0.137593
[31]	valid_0's l2: 0.0263415	valid_0's t1e: 0.136114
[32]	valid_0's l2: 0.0259216	valid_0's t1e: 0.134657
[33]	valid_0's l2: 0.025525	valid_0's t1e: 0.133274
[34]	valid_0's l2: 0.0251534	valid_0's t1e: 0.131948
[35]	valid_0's l2: 0.0247688	valid_0's t1e: 0.130567
[36]	valid_0's l2: 0.0244681	valid_0's t1e: 0.129454
[37]	valid_0's l2: 0.0241555	valid_0's t1e: 0.128266
[38]	valid_0's l2: 0.023858	valid_0's t1e: 0.127113
[39]	valid_0's l2: 0.0235902	valid_0's t1e: 0.126068
[40]	valid_0's l2: 0.0233549	valid_0's t1e: 0.125135
[41]	valid_0's l2: 0.0231131	valid_0's t1e: 0.124174
[42]	valid_0's l2: 0.0229011	valid_0's t1e: 0.123311
[43]	valid_0's l2: 0.0226779	valid_0's t1e: 0.122397
[44]	valid_0's l2: 0.0224733	valid_0's t1e: 0.121522
[45]	valid_0's l2: 0.022249	valid_0's t1e: 0.1206

[180]	valid_0's l2: 0.0174929	valid_0's t1e: 0.0990103
[181]	valid_0's l2: 0.0174915	valid_0's t1e: 0.0990098
[182]	valid_0's l2: 0.017492	valid_0's t1e: 0.099012
[183]	valid_0's l2: 0.0174846	valid_0's t1e: 0.0989918
[184]	valid_0's l2: 0.0174844	valid_0's t1e: 0.0989886
[185]	valid_0's l2: 0.0174793	valid_0's t1e: 0.098966
[186]	valid_0's l2: 0.0174746	valid_0's t1e: 0.098956
[187]	valid_0's l2: 0.0174711	valid_0's t1e: 0.0989443
[188]	valid_0's l2: 0.0174621	valid_0's t1e: 0.0989178
[189]	valid_0's l2: 0.0174631	valid_0's t1e: 0.0989204
[190]	valid_0's l2: 0.0174596	valid_0's t1e: 0.0989163
[191]	valid_0's l2: 0.0174575	valid_0's t1e: 0.098907
[192]	valid_0's l2: 0.017456	valid_0's t1e: 0.0988853
[193]	valid_0's l2: 0.0174539	valid_0's t1e: 0.0988719
[194]	valid_0's l2: 0.0174551	valid_0's t1e: 0.0988723
[195]	valid_0's l2: 0.0174527	valid_0's t1e: 0.0988678
[196]	valid_0's l2: 0.0174488	valid_0's t1e: 0.0988497
[197]	valid_0's l2: 0.0174489	valid_0's t1e: 0.0988501
[198]	valid_0's 

[76]	valid_0's l2: 0.018799	valid_0's t1e: 0.105089
[77]	valid_0's l2: 0.018727	valid_0's t1e: 0.104747
[78]	valid_0's l2: 0.0186927	valid_0's t1e: 0.104578
[79]	valid_0's l2: 0.0186345	valid_0's t1e: 0.104302
[80]	valid_0's l2: 0.0185897	valid_0's t1e: 0.10409
[81]	valid_0's l2: 0.0185504	valid_0's t1e: 0.103878
[82]	valid_0's l2: 0.0184978	valid_0's t1e: 0.103629
[83]	valid_0's l2: 0.0184536	valid_0's t1e: 0.103436
[84]	valid_0's l2: 0.0184045	valid_0's t1e: 0.103186
[85]	valid_0's l2: 0.0183817	valid_0's t1e: 0.103046
[86]	valid_0's l2: 0.0183502	valid_0's t1e: 0.102905
[87]	valid_0's l2: 0.0183061	valid_0's t1e: 0.102692
[88]	valid_0's l2: 0.0182575	valid_0's t1e: 0.102475
[89]	valid_0's l2: 0.0182362	valid_0's t1e: 0.10237
[90]	valid_0's l2: 0.0182096	valid_0's t1e: 0.10226
[91]	valid_0's l2: 0.0181725	valid_0's t1e: 0.102079
[92]	valid_0's l2: 0.0181392	valid_0's t1e: 0.101907
[93]	valid_0's l2: 0.0181191	valid_0's t1e: 0.101811
[94]	valid_0's l2: 0.0180922	valid_0's t1e: 0.10167

...

[111]	valid_0's l2: 9.23332e-05	valid_0's t1e: 0.0071756
[112]	valid_0's l2: 9.21296e-05	valid_0's t1e: 0.00716665
[113]	valid_0's l2: 9.19263e-05	valid_0's t1e: 0.00715631
[114]	valid_0's l2: 9.16999e-05	valid_0's t1e: 0.00714662
[115]	valid_0's l2: 9.15781e-05	valid_0's t1e: 0.0071408
[116]	valid_0's l2: 9.14188e-05	valid_0's t1e: 0.00713401
[117]	valid_0's l2: 9.12857e-05	valid_0's t1e: 0.00712933
[118]	valid_0's l2: 9.11336e-05	valid_0's t1e: 0.00712296
[119]	valid_0's l2: 9.09717e-05	valid_0's t1e: 0.00711666
[120]	valid_0's l2: 9.08309e-05	valid_0's t1e: 0.0071111
[121]	valid_0's l2: 9.07286e-05	valid_0's t1e: 0.00710649
[122]	valid_0's l2: 9.06471e-05	valid_0's t1e: 0.00710225
[123]	valid_0's l2: 9.05212e-05	valid_0's t1e: 0.00709746
[124]	valid_0's l2: 9.04558e-05	valid_0's t1e: 0.00709308
[125]	valid_0's l2: 9.02934e-05	valid_0's t1e: 0.00708802
[126]	valid_0's l2: 9.01765e-05	valid_0's t1e: 0.00708256
[127]	valid_0's l2: 9.00901e-05	valid_0's t1e: 0.00707842
[128]	valid_0's l

[253]	valid_0's l2: 8.64745e-05	valid_0's t1e: 0.00690988
[254]	valid_0's l2: 8.64793e-05	valid_0's t1e: 0.00690972
[255]	valid_0's l2: 8.6462e-05	valid_0's t1e: 0.00690921
[256]	valid_0's l2: 8.64595e-05	valid_0's t1e: 0.00690881
[257]	valid_0's l2: 8.64492e-05	valid_0's t1e: 0.0069085
[258]	valid_0's l2: 8.64556e-05	valid_0's t1e: 0.00690839
[259]	valid_0's l2: 8.64563e-05	valid_0's t1e: 0.00690838
[260]	valid_0's l2: 8.64458e-05	valid_0's t1e: 0.00690793
[261]	valid_0's l2: 8.64503e-05	valid_0's t1e: 0.00690794
[262]	valid_0's l2: 8.6413e-05	valid_0's t1e: 0.00690628
[263]	valid_0's l2: 8.63726e-05	valid_0's t1e: 0.00690502
[264]	valid_0's l2: 8.63649e-05	valid_0's t1e: 0.00690483
[265]	valid_0's l2: 8.63609e-05	valid_0's t1e: 0.00690453
[266]	valid_0's l2: 8.63491e-05	valid_0's t1e: 0.00690431
[267]	valid_0's l2: 8.6327e-05	valid_0's t1e: 0.00690348
[268]	valid_0's l2: 8.63171e-05	valid_0's t1e: 0.00690275
[269]	valid_0's l2: 8.63155e-05	valid_0's t1e: 0.00690222
[270]	valid_0's l2

[395]	valid_0's l2: 8.52542e-05	valid_0's t1e: 0.00684943
[396]	valid_0's l2: 8.52354e-05	valid_0's t1e: 0.0068485
[397]	valid_0's l2: 8.52228e-05	valid_0's t1e: 0.00684796
[398]	valid_0's l2: 8.52046e-05	valid_0's t1e: 0.00684771
[399]	valid_0's l2: 8.5182e-05	valid_0's t1e: 0.0068464
[400]	valid_0's l2: 8.51797e-05	valid_0's t1e: 0.00684623
[401]	valid_0's l2: 8.51895e-05	valid_0's t1e: 0.00684618
[402]	valid_0's l2: 8.51858e-05	valid_0's t1e: 0.0068458
[403]	valid_0's l2: 8.51753e-05	valid_0's t1e: 0.00684542
[404]	valid_0's l2: 8.5168e-05	valid_0's t1e: 0.0068447
[405]	valid_0's l2: 8.51539e-05	valid_0's t1e: 0.00684397
[406]	valid_0's l2: 8.51408e-05	valid_0's t1e: 0.00684299
[407]	valid_0's l2: 8.51283e-05	valid_0's t1e: 0.0068426
[408]	valid_0's l2: 8.51369e-05	valid_0's t1e: 0.00684283
[409]	valid_0's l2: 8.51368e-05	valid_0's t1e: 0.00684258
[410]	valid_0's l2: 8.51151e-05	valid_0's t1e: 0.00684142
[411]	valid_0's l2: 8.513e-05	valid_0's t1e: 0.00684162
[412]	valid_0's l2: 8.5

[94]	valid_0's l2: 9.49451e-05	valid_0's t1e: 0.00731736
[95]	valid_0's l2: 9.44215e-05	valid_0's t1e: 0.00729726
[96]	valid_0's l2: 9.39596e-05	valid_0's t1e: 0.00727943
[97]	valid_0's l2: 9.35133e-05	valid_0's t1e: 0.00726295
[98]	valid_0's l2: 9.30582e-05	valid_0's t1e: 0.00724535
[99]	valid_0's l2: 9.266e-05	valid_0's t1e: 0.00723003
[100]	valid_0's l2: 9.23298e-05	valid_0's t1e: 0.0072176
[101]	valid_0's l2: 9.19996e-05	valid_0's t1e: 0.00720468
[102]	valid_0's l2: 9.16974e-05	valid_0's t1e: 0.00719331
[103]	valid_0's l2: 9.14002e-05	valid_0's t1e: 0.00718061
[104]	valid_0's l2: 9.11097e-05	valid_0's t1e: 0.00717027
[105]	valid_0's l2: 9.0829e-05	valid_0's t1e: 0.00715832
[106]	valid_0's l2: 9.0563e-05	valid_0's t1e: 0.00714762
[107]	valid_0's l2: 9.02577e-05	valid_0's t1e: 0.00713718
[108]	valid_0's l2: 9.00344e-05	valid_0's t1e: 0.00712841
[109]	valid_0's l2: 8.98428e-05	valid_0's t1e: 0.00712185
[110]	valid_0's l2: 8.95943e-05	valid_0's t1e: 0.00711135
[111]	valid_0's l2: 8.932

[236]	valid_0's l2: 8.2828e-05	valid_0's t1e: 0.00682712
[237]	valid_0's l2: 8.28138e-05	valid_0's t1e: 0.00682635
[238]	valid_0's l2: 8.28023e-05	valid_0's t1e: 0.00682619
[239]	valid_0's l2: 8.27813e-05	valid_0's t1e: 0.00682526
[240]	valid_0's l2: 8.27276e-05	valid_0's t1e: 0.00682335
[241]	valid_0's l2: 8.27196e-05	valid_0's t1e: 0.00682324
[242]	valid_0's l2: 8.27161e-05	valid_0's t1e: 0.00682271
[243]	valid_0's l2: 8.2707e-05	valid_0's t1e: 0.00682265
[244]	valid_0's l2: 8.26899e-05	valid_0's t1e: 0.00682184
[245]	valid_0's l2: 8.26739e-05	valid_0's t1e: 0.00682116
[246]	valid_0's l2: 8.26661e-05	valid_0's t1e: 0.00682063
[247]	valid_0's l2: 8.26669e-05	valid_0's t1e: 0.00682063
[248]	valid_0's l2: 8.26374e-05	valid_0's t1e: 0.00681943
[249]	valid_0's l2: 8.26227e-05	valid_0's t1e: 0.00681867
[250]	valid_0's l2: 8.25972e-05	valid_0's t1e: 0.00681764
[251]	valid_0's l2: 8.25874e-05	valid_0's t1e: 0.00681709
[252]	valid_0's l2: 8.25989e-05	valid_0's t1e: 0.00681752
[253]	valid_0's 

In [13]:
ty = []
vy = []
feature_imp = []

for y_id in range(3):
    kf = KFold(n_splits=5, shuffle=False)
    tty = []
    tvy = []
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = Y[train_idx, y_id], Y[val_idx, y_id]
        
        model = lgb.LGBMRegressor(**params)
        model.fit(X_train, y_train,  
                  eval_set=[(X_val, y_val)], 
                  eval_metric=scorer['t2'], 
                  early_stopping_rounds=5)
        feature_imp.append(model.feature_importances_)
        
        tvy.append(model.predict(X_val))
        tty.append(model.predict(X_test))
        
        X_train, X_val, y_train, y_val = None, None, None, None
    vy.append(np.concatenate((tvy[0], tvy[1], tvy[2], tvy[3], tvy[4]), axis=0))
    ty.append(np.mean(np.array(tty), axis=0))

ty = np.array(ty).transpose()
vy = np.array(vy).transpose()
df = pd.DataFrame(ty)
df.to_csv('t2_test.csv', index=False, header=False)
df = pd.DataFrame(vy)
df.to_csv('t2_val.csv', index=False, header=False)
feature_imp = np.array(feature_imp).transpose()
df = pd.DataFrame(feature_imp)
df.to_csv('feature_imp2.csv', index=False, header=False)

[1]	valid_0's l2: 0.0797814	valid_0's t2e: 3.61821
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 0.0750526	valid_0's t2e: 3.50384
[3]	valid_0's l2: 0.070744	valid_0's t2e: 3.38851
[4]	valid_0's l2: 0.0668024	valid_0's t2e: 3.27726
[5]	valid_0's l2: 0.0631663	valid_0's t2e: 3.17101
[6]	valid_0's l2: 0.0598537	valid_0's t2e: 3.07454
[7]	valid_0's l2: 0.0568177	valid_0's t2e: 2.98099
[8]	valid_0's l2: 0.0540238	valid_0's t2e: 2.89158
[9]	valid_0's l2: 0.0515034	valid_0's t2e: 2.80748
[10]	valid_0's l2: 0.0491692	valid_0's t2e: 2.72676
[11]	valid_0's l2: 0.0469983	valid_0's t2e: 2.64848
[12]	valid_0's l2: 0.045037	valid_0's t2e: 2.57315
[13]	valid_0's l2: 0.0431888	valid_0's t2e: 2.50085
[14]	valid_0's l2: 0.0415124	valid_0's t2e: 2.43129
[15]	valid_0's l2: 0.0399797	valid_0's t2e: 2.37005
[16]	valid_0's l2: 0.038583	valid_0's t2e: 2.30975
[17]	valid_0's l2: 0.0373024	valid_0's t2e: 2.24861
[18]	valid_0's l2: 0.0360864	valid_0's t2e: 2.18938
[19]	valid_0's 

[7]	valid_0's l2: 0.0555752	valid_0's t2e: 3.20211
[8]	valid_0's l2: 0.0528297	valid_0's t2e: 3.10176
[9]	valid_0's l2: 0.0503783	valid_0's t2e: 3.01138
[10]	valid_0's l2: 0.0481146	valid_0's t2e: 2.92154
[11]	valid_0's l2: 0.0460299	valid_0's t2e: 2.83672
[12]	valid_0's l2: 0.0441229	valid_0's t2e: 2.759
[13]	valid_0's l2: 0.0423491	valid_0's t2e: 2.68318
[14]	valid_0's l2: 0.040774	valid_0's t2e: 2.60937
[15]	valid_0's l2: 0.0392768	valid_0's t2e: 2.54099
[16]	valid_0's l2: 0.0378942	valid_0's t2e: 2.46738
[17]	valid_0's l2: 0.0366281	valid_0's t2e: 2.39849
[18]	valid_0's l2: 0.0354732	valid_0's t2e: 2.33378
[19]	valid_0's l2: 0.0344127	valid_0's t2e: 2.27598
[20]	valid_0's l2: 0.0334456	valid_0's t2e: 2.21489
[21]	valid_0's l2: 0.0325177	valid_0's t2e: 2.15921
[22]	valid_0's l2: 0.0316897	valid_0's t2e: 2.10722
[23]	valid_0's l2: 0.0308927	valid_0's t2e: 2.05742
[24]	valid_0's l2: 0.0302025	valid_0's t2e: 2.009
[25]	valid_0's l2: 0.0295424	valid_0's t2e: 1.96532
[26]	valid_0's l2: 0

[6]	valid_0's l2: 0.0584365	valid_0's t2e: 3.41248
[7]	valid_0's l2: 0.055491	valid_0's t2e: 3.29613
[8]	valid_0's l2: 0.0527546	valid_0's t2e: 3.17984
[9]	valid_0's l2: 0.0503113	valid_0's t2e: 3.07967
[10]	valid_0's l2: 0.0480385	valid_0's t2e: 2.97795
[11]	valid_0's l2: 0.0459961	valid_0's t2e: 2.8814
[12]	valid_0's l2: 0.0440655	valid_0's t2e: 2.79319
[13]	valid_0's l2: 0.0423033	valid_0's t2e: 2.70461
[14]	valid_0's l2: 0.0407033	valid_0's t2e: 2.61959
[15]	valid_0's l2: 0.0392217	valid_0's t2e: 2.53504
[16]	valid_0's l2: 0.0378825	valid_0's t2e: 2.45645
[17]	valid_0's l2: 0.0365945	valid_0's t2e: 2.38665
[18]	valid_0's l2: 0.0354174	valid_0's t2e: 2.31412
[19]	valid_0's l2: 0.0343586	valid_0's t2e: 2.25072
[20]	valid_0's l2: 0.0333823	valid_0's t2e: 2.18422
[21]	valid_0's l2: 0.0324863	valid_0's t2e: 2.12633
[22]	valid_0's l2: 0.0316414	valid_0's t2e: 2.07074
[23]	valid_0's l2: 0.0308348	valid_0's t2e: 2.01359
[24]	valid_0's l2: 0.0301113	valid_0's t2e: 1.9608
[25]	valid_0's l2: 

[23]	valid_0's l2: 0.0310432	valid_0's t2e: 2.52489
[24]	valid_0's l2: 0.0303085	valid_0's t2e: 2.4514
[25]	valid_0's l2: 0.0296278	valid_0's t2e: 2.38399
[26]	valid_0's l2: 0.0290237	valid_0's t2e: 2.31763
[27]	valid_0's l2: 0.0284494	valid_0's t2e: 2.24839
[28]	valid_0's l2: 0.0279075	valid_0's t2e: 2.18687
[29]	valid_0's l2: 0.0274125	valid_0's t2e: 2.12974
[30]	valid_0's l2: 0.0269581	valid_0's t2e: 2.07417
[31]	valid_0's l2: 0.0265044	valid_0's t2e: 2.01506
[32]	valid_0's l2: 0.0260801	valid_0's t2e: 1.96698
[33]	valid_0's l2: 0.0256598	valid_0's t2e: 1.91956
[34]	valid_0's l2: 0.025317	valid_0's t2e: 1.86915
[35]	valid_0's l2: 0.0249855	valid_0's t2e: 1.82318
[36]	valid_0's l2: 0.0246678	valid_0's t2e: 1.7738
[37]	valid_0's l2: 0.0243782	valid_0's t2e: 1.73858
[38]	valid_0's l2: 0.024083	valid_0's t2e: 1.69143
[39]	valid_0's l2: 0.0237872	valid_0's t2e: 1.65489
[40]	valid_0's l2: 0.0235359	valid_0's t2e: 1.6202
[41]	valid_0's l2: 0.0232755	valid_0's t2e: 1.58078
[42]	valid_0's l2

[71]	valid_0's l2: 0.0197317	valid_0's t2e: 0.880145
[72]	valid_0's l2: 0.0196377	valid_0's t2e: 0.86741
[73]	valid_0's l2: 0.0195753	valid_0's t2e: 0.859929
[74]	valid_0's l2: 0.0194913	valid_0's t2e: 0.84732
[75]	valid_0's l2: 0.0194347	valid_0's t2e: 0.839946
[76]	valid_0's l2: 0.0193697	valid_0's t2e: 0.831521
[77]	valid_0's l2: 0.0192961	valid_0's t2e: 0.820792
[78]	valid_0's l2: 0.0192453	valid_0's t2e: 0.81052
[79]	valid_0's l2: 0.0191965	valid_0's t2e: 0.80527
[80]	valid_0's l2: 0.0191627	valid_0's t2e: 0.801902
[81]	valid_0's l2: 0.019103	valid_0's t2e: 0.793997
[82]	valid_0's l2: 0.0190525	valid_0's t2e: 0.787261
[83]	valid_0's l2: 0.0189875	valid_0's t2e: 0.778086
[84]	valid_0's l2: 0.0189364	valid_0's t2e: 0.770514
[85]	valid_0's l2: 0.0188892	valid_0's t2e: 0.763396
[86]	valid_0's l2: 0.0188594	valid_0's t2e: 0.761643
[87]	valid_0's l2: 0.0188282	valid_0's t2e: 0.754989
[88]	valid_0's l2: 0.0187821	valid_0's t2e: 0.746779
[89]	valid_0's l2: 0.0187528	valid_0's t2e: 0.74268

...

[136]	valid_0's l2: 8.92257e-05	valid_0's t2e: 0.00984085
[137]	valid_0's l2: 8.91584e-05	valid_0's t2e: 0.00983498
[138]	valid_0's l2: 8.9082e-05	valid_0's t2e: 0.00983093
[139]	valid_0's l2: 8.89876e-05	valid_0's t2e: 0.00982454
[140]	valid_0's l2: 8.89054e-05	valid_0's t2e: 0.00981818
[141]	valid_0's l2: 8.88532e-05	valid_0's t2e: 0.00981403
[142]	valid_0's l2: 8.88011e-05	valid_0's t2e: 0.00980917
[143]	valid_0's l2: 8.8757e-05	valid_0's t2e: 0.00980681
[144]	valid_0's l2: 8.86959e-05	valid_0's t2e: 0.00980364
[145]	valid_0's l2: 8.86582e-05	valid_0's t2e: 0.0098001
[146]	valid_0's l2: 8.86137e-05	valid_0's t2e: 0.00979759
[147]	valid_0's l2: 8.85408e-05	valid_0's t2e: 0.00979207
[148]	valid_0's l2: 8.85222e-05	valid_0's t2e: 0.00978791
[149]	valid_0's l2: 8.84926e-05	valid_0's t2e: 0.00978494
[150]	valid_0's l2: 8.84662e-05	valid_0's t2e: 0.00978404
[151]	valid_0's l2: 8.84291e-05	valid_0's t2e: 0.00978019
[152]	valid_0's l2: 8.8399e-05	valid_0's t2e: 0.00977818
[153]	valid_0's l2

[278]	valid_0's l2: 8.62218e-05	valid_0's t2e: 0.00962757
[279]	valid_0's l2: 8.62151e-05	valid_0's t2e: 0.00962673
[280]	valid_0's l2: 8.61948e-05	valid_0's t2e: 0.00962477
[281]	valid_0's l2: 8.6194e-05	valid_0's t2e: 0.00962524
[282]	valid_0's l2: 8.61774e-05	valid_0's t2e: 0.00962418
[283]	valid_0's l2: 8.6191e-05	valid_0's t2e: 0.00962443
[284]	valid_0's l2: 8.61783e-05	valid_0's t2e: 0.00962359
[285]	valid_0's l2: 8.6157e-05	valid_0's t2e: 0.00962211
[286]	valid_0's l2: 8.61476e-05	valid_0's t2e: 0.00962134
[287]	valid_0's l2: 8.61437e-05	valid_0's t2e: 0.00962096
[288]	valid_0's l2: 8.61536e-05	valid_0's t2e: 0.00962072
[289]	valid_0's l2: 8.61232e-05	valid_0's t2e: 0.00961818
[290]	valid_0's l2: 8.61143e-05	valid_0's t2e: 0.00961743
[291]	valid_0's l2: 8.60978e-05	valid_0's t2e: 0.00961612
[292]	valid_0's l2: 8.61124e-05	valid_0's t2e: 0.00961641
[293]	valid_0's l2: 8.60871e-05	valid_0's t2e: 0.00961538
[294]	valid_0's l2: 8.60712e-05	valid_0's t2e: 0.00961471
[295]	valid_0's l

[420]	valid_0's l2: 8.50772e-05	valid_0's t2e: 0.0095413
[421]	valid_0's l2: 8.50602e-05	valid_0's t2e: 0.00953999
[422]	valid_0's l2: 8.50382e-05	valid_0's t2e: 0.00953904
[423]	valid_0's l2: 8.50364e-05	valid_0's t2e: 0.00953956
[424]	valid_0's l2: 8.5033e-05	valid_0's t2e: 0.00953935
[425]	valid_0's l2: 8.50161e-05	valid_0's t2e: 0.00953819
[426]	valid_0's l2: 8.50163e-05	valid_0's t2e: 0.0095382
[427]	valid_0's l2: 8.5013e-05	valid_0's t2e: 0.00953746
[428]	valid_0's l2: 8.50109e-05	valid_0's t2e: 0.00953719
[429]	valid_0's l2: 8.49821e-05	valid_0's t2e: 0.00953558
[430]	valid_0's l2: 8.49783e-05	valid_0's t2e: 0.00953575
[431]	valid_0's l2: 8.49823e-05	valid_0's t2e: 0.00953543
[432]	valid_0's l2: 8.49739e-05	valid_0's t2e: 0.00953476
[433]	valid_0's l2: 8.49717e-05	valid_0's t2e: 0.00953442
[434]	valid_0's l2: 8.49701e-05	valid_0's t2e: 0.00953466
[435]	valid_0's l2: 8.49607e-05	valid_0's t2e: 0.0095344
[436]	valid_0's l2: 8.49574e-05	valid_0's t2e: 0.00953417
[437]	valid_0's l2:

[120]	valid_0's l2: 8.75844e-05	valid_0's t2e: 0.00983406
[121]	valid_0's l2: 8.74614e-05	valid_0's t2e: 0.00982836
[122]	valid_0's l2: 8.7319e-05	valid_0's t2e: 0.00981931
[123]	valid_0's l2: 8.71605e-05	valid_0's t2e: 0.0098075
[124]	valid_0's l2: 8.70357e-05	valid_0's t2e: 0.00980139
[125]	valid_0's l2: 8.69788e-05	valid_0's t2e: 0.00979657
[126]	valid_0's l2: 8.68609e-05	valid_0's t2e: 0.00979079
[127]	valid_0's l2: 8.67213e-05	valid_0's t2e: 0.00978249
[128]	valid_0's l2: 8.66166e-05	valid_0's t2e: 0.00977725
[129]	valid_0's l2: 8.65283e-05	valid_0's t2e: 0.00977248
[130]	valid_0's l2: 8.64323e-05	valid_0's t2e: 0.0097656
[131]	valid_0's l2: 8.63089e-05	valid_0's t2e: 0.00975744
[132]	valid_0's l2: 8.61765e-05	valid_0's t2e: 0.00974699
[133]	valid_0's l2: 8.60961e-05	valid_0's t2e: 0.00974249
[134]	valid_0's l2: 8.60201e-05	valid_0's t2e: 0.00973721
[135]	valid_0's l2: 8.5903e-05	valid_0's t2e: 0.00973057
[136]	valid_0's l2: 8.57761e-05	valid_0's t2e: 0.00972157
[137]	valid_0's l2

[262]	valid_0's l2: 8.25298e-05	valid_0's t2e: 0.00951224
[263]	valid_0's l2: 8.25143e-05	valid_0's t2e: 0.00951154
[264]	valid_0's l2: 8.24925e-05	valid_0's t2e: 0.00950779
[265]	valid_0's l2: 8.24802e-05	valid_0's t2e: 0.00950697
[266]	valid_0's l2: 8.24863e-05	valid_0's t2e: 0.00950746
[267]	valid_0's l2: 8.24795e-05	valid_0's t2e: 0.00950649
[268]	valid_0's l2: 8.24756e-05	valid_0's t2e: 0.00950603
[269]	valid_0's l2: 8.24526e-05	valid_0's t2e: 0.00950559
[270]	valid_0's l2: 8.24544e-05	valid_0's t2e: 0.00950586
[271]	valid_0's l2: 8.24506e-05	valid_0's t2e: 0.00950551
[272]	valid_0's l2: 8.24631e-05	valid_0's t2e: 0.00950612
[273]	valid_0's l2: 8.24429e-05	valid_0's t2e: 0.00950456
[274]	valid_0's l2: 8.2437e-05	valid_0's t2e: 0.00950361
[275]	valid_0's l2: 8.24381e-05	valid_0's t2e: 0.00950373
[276]	valid_0's l2: 8.24211e-05	valid_0's t2e: 0.00950228
[277]	valid_0's l2: 8.24059e-05	valid_0's t2e: 0.00950136
[278]	valid_0's l2: 8.2394e-05	valid_0's t2e: 0.00950051
[279]	valid_0's 

In [22]:
df = pd.read_csv('t1_val.csv', header=None)

In [26]:
err1(np.array(df)[:, 0], Y[:, 0])[1] * 300

29.530289088188024

In [27]:
err1(np.array(df)[:, 1], Y[:, 1])[1] * 1

20.231177530843958

In [28]:
err1(np.array(df)[:, 2], Y[:, 2])[1] * 200

1.362652541774901